## DERIVATIVES PRICER 
* Code is centered around tree-pricing models and monte carlo simulations

### TO DO: 
* Add vectorised MC 
* Add tree methods
* Add variance reduction methods to MC
* Show how non-closed solution american options can be priced from the base of the asian MC sim
* Add visuals for demonstrating the impact of IV on pricing
* Add GARCH models for volatility forecasting
* Price and backtest real traded options to see the accuracy of said models in practice (ex. liquidity constraints)

In [1]:
## import packages
from scipy.stats import norm
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

In [8]:
def bmsim(T, N, X0=0, mu=0, sigma=1):
    """Simulate a Brownian motion path with X0/mu = 0 for drift terms"""
    
    deltaT = float(T)/N
    
    #N+1 is one more than we need, actually. This way we won't have to grow dX by X0.
    tvec = np.linspace(0, T, N+1)
    z = np.random.randn(N+1)   
    dX = mu*deltaT + sigma*np.sqrt(deltaT)*z  #X[j+1]-X[j]=mu*deltaT + sigma*np.sqrt(deltaT)*z[j].
    dX[0] = 0.
    X = np.cumsum(dX)
    X += X0    
    return tvec, X

In [9]:

def asianmc(S0, K, T, r, sigma, q, N, numsim=10000):
    """Monte Carlo price of an arithmetic average Asian call."""
    X0 = np.log(S0)
    nu = r-q-.5*sigma**2
    payoffs = np.zeros(numsim)
    for i in range(numsim):
        _, X = bmsim(T, N, X0, nu, sigma)  #Convention: underscore holds value to be discarded.
        S = np.exp(X)
        payoffs[i] = max(S[1:].mean()-K, 0.)  #We ignore S0 in the mean.
    g = np.exp(-r*T)*payoffs
    C = g.mean(); s = g.std()
    zq = norm.ppf(0.975)
    Cl = C-(zq*s)/np.sqrt(numsim)
    Cu = C+(zq*s)/np.sqrt(numsim)
    return C, Cl, Cu

In [10]:
%%time

S0 = 11; K = 10; T = 3/12.; r = 0.02; sigma = .3; q = 0.01; N = 10

# ensure random seed for comparability
np.random.seed(0)

C0, Cl, Cu = asianmc(S0, K, T, r, sigma, q, N); C0, Cl, Cu

Wall time: 511 ms


(1.0927262054551385, 1.0747653929130998, 1.1106870179971773)

In [ ]:
## vectorise the BM paths

In [5]:
#Note new input: numsim, the number of paths.
## check -- error in code when plugging into MC! 

def bmsim_vec(T, N, X0=0, mu=0, sigma=1, numsim=1):  
    """Simulate `numsim` Brownian motion paths."""
    deltaT = float(T)/N
    tvec = np.linspace(0, T, N+1)
    z = np.random.randn(numsim, N+1)  
    dX = mu*deltaT + sigma*np.sqrt(deltaT)*z
    
    dX[:, 0] = 0.  
    X = np.cumsum(dX, axis=1)  
    
    X += X0    
    return tvec, X